# Models

In [ ]:
import os

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

# 1. langchain 的 init_chat_model 函数
# model_provider 是 LangChain 在 v1.0 之后为了“解耦模型名称与供应商”而引入的
model = init_chat_model(model_provider="openai", model="gpt-3.5-turbo")  # 

response = model.invoke("为什么鹦鹉会说话？")

print(response)
print(response.content)

In [ ]:
import os
from langchain_openai import ChatOpenAI

# 2. langchain_openai 的 ChatOpenAI 类
model = ChatOpenAI(model="gpt-3.5-turbo")

response = model.invoke("如何学习 LangChain")

print(response)

print(response.content)

## 模型传入的参数

- model
- api_key
- temperature
- max_tokens
- max-retries



## 1. 调用

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
  "openai:gpt-3.5-turbo",
  # Kwargs passed to the model:
  temperature=0.7,
  timeout=30,
  max_tokens=1000,
)
response = model.invoke("为什么鹦鹉的羽毛颜色鲜艳？")
print(response.content)


In [ ]:
# 字典格式
from langchain.messages import HumanMessage, AIMessage, SystemMessage

conversation = [
  {"role": "system", "content": "You are a helpful assistant that translates Chinese to English."},
  {"role": "user", "content": "翻译：我热爱编程"},
  {"role": "assistant", "content": "I love programming."},
  {"role": "user", "content": "翻译：我喜欢构建应用程序"}
]

response = model.invoke(conversation)
print(response)  # AIMessage("I enjoy building applications.")

In [ ]:
# 消息格式
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

conversation = [
  SystemMessage(content="You are a helpful assistant that translates Chinese to English."),
  HumanMessage(content="翻译：我热爱编程"),
  AIMessage(content="I love programming."),
  HumanMessage(content="翻译：我喜欢构建应用程序")
]

response = model.invoke(conversation)
print(response)  # AIMessage("I enjoy building applications.")

### 流式传输

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
  "openai:gpt-3.5-turbo",
  streaming=True,
)

for chunk in model.stream("为什么鹦鹉的羽毛颜色鲜艳？"):
  print(chunk.text, end="", flush=True)

In [ ]:
full = None  # None | AIMessageChunk
for chunk in model.stream("What color is the sky?"):
  full = chunk if full is None else full + chunk
  print(full.text)

# The
# The sky
# The sky is
# The sky is typically
# The sky is typically blue
# ...

print(full.content)
# Usually blue during the day. That blue comes from Rayleigh scattering: sunlight hits the atmosphere and the shorter blue wavelengths scatter in all directions. The color can change: red/orange/pink at sunrise and sunset, gray on cloudy days, and dark at night (with stars visible). Want the answer for a specific time or place?

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.tools import tool


@tool
def get_weather(city: str) -> str:
  """Get the weather for a city."""
  return f"The weather in {city} is sunny."


model = ChatOpenAI(model="gpt-4o-mini")

agent = create_agent(
  model=model,
  tools=[get_weather],
)

# 🌊 实时流式输出
for chunk in agent.stream(
  {"messages": [{"role": "user", "content": "查下深圳天气，然后写首诗"}]},
  stream_mode="values",  # 输出 AgentState 中的每个阶段
):
  latest_msg = chunk["messages"][-1]
  if latest_msg.content:
    print(latest_msg.content, end="", flush=True)


### 批量 Batch

In [ ]:
# 返回整个批次的最终输出
responses = model.batch([
  "Why do parrots have colorful feathers?",
  "How do airplanes fly?",
  "What is quantum computing?"
])
for response in responses:
  print(response.content)

In [ ]:
# 逐个接收每个输入生成完成时的输出.在使用 batch_as_completed() 时，结果可能会乱序到达。每个结果都包含输入索引，以便在需要时匹配以重建原始顺序。
for response in model.batch_as_completed([
  "为什么鹦鹉会说话？",
  "如何飞机飞行？",
  "什么是量子计算？"
], config={"batch_size": 1, "max_concurrency": 2}):  # 批量大小为1，并发数为2

  print(response)

## 2.工具调用

In [ ]:
from langchain.tools import tool
from langchain.chat_models import init_chat_model


@tool
def get_weather(location: str) -> str:
  """Get the weather at a location."""
  return f"It's sunny in {location}."


model = init_chat_model(
  "openai:gpt-3.5-turbo",
  streaming=True,
)

model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")
for tool_call in response.tool_calls:
  # View tool calls made by the model
  print(f"Tool: {tool_call['name']}")
  print(f"Args: {tool_call['args']}")

response.pretty_print()

## 2.1 工具执行循环

In [ ]:
# 将（可能多个）工具绑定到模型
model_with_tools = model.bind_tools([get_weather])

# Step 1: 模型生成工具调用
messages = [{"role": "user", "content": "今天波士顿的天气如何？"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

print(messages)

# Step 2: 执行工具并收集结果
for tool_call in ai_msg.tool_calls:
  # 使用生成的参数执行该工具
  tool_result = get_weather.invoke(tool_call)
  messages.append(tool_result)

print(messages)

# Step 3: 将结果传回模型以获得最终响应
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

final_response.pretty_print()

2.2 强制工具调用

强制使用任何工具： 
```python
model_with_tools = model.bind_tools([tool_1], tool_choice="any")
```

强制使用特定工具
```python
model_with_tools = model.bind_tools([tool_1], tool_choice="tool_1")
```

### 2.3 并行工具调用


In [ ]:
model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke(
  "What's the weather in Boston and Tokyo?"
)

# The model may generate multiple tool calls
print(response.tool_calls)
# [
#   {'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_1'},
#   {'name': 'get_weather', 'args': {'location': 'Tokyo'}, 'id': 'call_2'}
# ]


# Execute all tools (can be done in parallel with async)
results = []
for tool_call in response.tool_calls:
  if tool_call['name'] == 'get_weather':
    result = get_weather.invoke(tool_call)
  # ...
  results.append(result)

print(results)

大多数支持工具调用的模型默认启用并行工具调用。一些模型（包括 OpenAI 和 Anthropic）允许您禁用此功能。要做到这一点，请设置 parallel_tool_calls=False
```python
model.bind_tools([get_weather], parallel_tool_calls=False)
```


### 2.4 流式工具调用

In [ ]:
for chunk in model_with_tools.stream(
  "What's the weather in Boston and Tokyo?"
):
  # Tool call chunks arrive progressively
  for tool_chunk in chunk.tool_call_chunks:
    if name := tool_chunk.get("name"):
      print(f"Tool: {name}")
    if id_ := tool_chunk.get("id"):
      print(f"ID: {id_}")
    if args := tool_chunk.get("args"):
      print(f"Args: {args}")

# Output:
# Tool: get_weather
# ID: call_SvMlU1TVIZugrFLckFE2ceRE
# Args: {"lo
# Args: catio
# Args: n": "B
# Args: osto
# Args: n"}
# Tool: get_weather
# ID: call_QMZdy6qInx13oWKE7KhuhOLR
# Args: {"lo
# Args: catio
# Args: n": "T
# Args: okyo
# Args: "}

In [ ]:
gathered = None
for chunk in model_with_tools.stream("What's the weather in Boston?"):
  gathered = chunk if gathered is None else gathered + chunk
  print(gathered)

## 3.结构树输出

In [ ]:
# 3.1 Pydantic 模型 提供了最丰富的功能集，包括字段验证、描述和嵌套结构。
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model


class Movie(BaseModel):
  """A movie with details."""
  title: str = Field(..., description="The title of the movie")
  year: int = Field(..., description="The year the movie was released")
  director: str = Field(..., description="The director of the movie")
  rating: float = Field(..., description="The movie's rating out of 10")


model = init_chat_model("openai:gpt-5")

# include_raw=True 会在响应中包含原始文本，便于调试和记录。
model_with_structure = model.with_structured_output(Movie, include_raw=True)

response = model_with_structure.invoke("提供电影《Inception》的信息。")

print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

In [ ]:
# 3.2 TypedDict 提供了一种更简单的替代方案，使用 Python 内置的类型，适合在不需要运行时验证的情况下使用。

from typing_extensions import TypedDict, Annotated


class MovieDict(TypedDict):
  """A movie with details."""
  title: Annotated[str, ..., "The title of the movie"]
  year: Annotated[int, ..., "The year the movie was released"]
  director: Annotated[str, ..., "The director of the movie"]
  rating: Annotated[float, ..., "The movie's rating out of 10"]


model_with_structure = model.with_structured_output(MovieDict)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # {'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}

In [ ]:
# 3.3 为了获得最大的控制权或互操作性，您可以提供原始 JSON Schema。

import json

json_schema = {
  "title": "Movie",
  "description": "A movie with details",
  "type": "object",
  "properties": {
    "title": {
      "type": "string",
      "description": "The title of the movie"
    },
    "year": {
      "type": "integer",
      "description": "The year the movie was released"
    },
    "director": {
      "type": "string",
      "description": "The director of the movie"
    },
    "rating": {
      "type": "number",
      "description": "The movie's rating out of 10"
    }
  },
  "required": ["title", "year", "director", "rating"]
}

model_with_structure = model.with_structured_output(
  json_schema,
  method="json_schema",
)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # {'title': 'Inception', 'year': 2010, ...}

In [ ]:
# 嵌套结构
from pydantic import BaseModel, Field


class Actor(BaseModel):
  name: str
  role: str


class MovieDetails(BaseModel):
  title: str
  year: int
  cast: list[Actor]
  genres: list[str]
  budget: float | None = Field(None, description="预算（单位：百万美元）")


model_with_structure = model.with_structured_output(MovieDetails)

response = model_with_structure.invoke("使用中文提供电影《Inception》的信息")
print(response)

## 4. 高级配置

### 4.1 多模态

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5")

response = model.invoke("Create a picture of a cat use base64")
print(response.content)

### 4.2 推理

In [ ]:
for chunk in model.stream("人生的意义是什么？"):
  print(chunk.content)

In [ ]:
response = model.invoke("人生的意义是什么？")
reasoning_steps = [b for b in response.content_blocks if b["type"] == "reasoning"]
print(" ".join(step["reasoning"] for step in reasoning_steps))

## 4.3 Ollama 模型示例

In [ ]:
from langchain_ollama import ChatOllama

# 初始化本地模型
model = ChatOllama(
  model="deepseek-r1:8b",  # 模型名称与 ollama list 中一致
  temperature=0.7,  # 可选参数
)

# 调用模型
response = model.invoke("请用中文解释 Transformer 的自注意力机制。")
print(response.content)


## 4.3 高级配置 - 缓存

In [ ]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache

set_llm_cache(InMemoryCache())

response = model.invoke("Tell me a joke")
print(response)


In [ ]:
response = model.invoke("Tell me a joke")  # Fast, from cache
print(response)

### 4.4 服务器端工具使用

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-4o")  ## 经测试 gpt-4o-tools 不提供 web_search 工具, 可能是代理服务器问题

tool = {"type": "web_search"}
model_with_tools = model.bind_tools([tool])

response = model_with_tools.invoke("What was a positive news story from today?")
response.content_blocks

### 4.5 高级配置 - 速率限制

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
  requests_per_second=0.1,  # 表示每秒只能发 0.1 次请求，等价于 10 秒允许 1 次请求。
  check_every_n_seconds=0.1,  # 限流器会每 0.1 秒（100 毫秒）检查一次，看看是不是可以放行请求
  max_bucket_size=10,  # 采用令牌桶算法（token bucket）。允许一次性“积攒”最多 10 个请求作为突发流量。例如你空闲了 100 秒没请求，就能一次性发 10 个，但不会无限积累。
)

model = init_chat_model(
  model="gpt-5",
  model_provider="openai",
  rate_limiter=rate_limiter
)

### 4.6 基础 URL 或代理

In [ ]:
model = init_chat_model(
  model="MODEL_NAME",
  model_provider="openai",
  base_url="BASE_URL",
  api_key="YOUR_API_KEY",
)

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
  model="gpt-4o",
  openai_proxy="http://proxy.example.com:8080",
  api_key="sk_xxx",
)

### 4.7 日志概率

某些模型可以通过在初始化模型时设置 **logprobs** 参数，来配置返回 **token 级别的对数概率（log probabilities）**，用于表示每个给定 token 的可能性。


In [ ]:
model = init_chat_model(
  model="gpt-4o",
  model_provider="openai"
).bind(logprobs=True)

response = model.invoke("人生的意义是什么？")
print(response.response_metadata["logprobs"])

### 4.8 高级配置 - 令牌使用

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

model_1 = init_chat_model(model="openai:gpt-4o-mini")
model_2 = init_chat_model(model="openai:gpt-5")

callback = UsageMetadataCallbackHandler()
result_1 = model_1.invoke("Hello", config={"callbacks": [callback]})
result_2 = model_2.invoke("Hello", config={"callbacks": [callback]})
callback.usage_metadata

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import get_usage_metadata_callback

model_1 = init_chat_model(model="openai:gpt-4o-mini")
model_2 = init_chat_model(model="openai:gpt-5")

with get_usage_metadata_callback() as cb:
  model_1.invoke("Hello")
  model_2.invoke("Hello")
  print(cb.usage_metadata)

### 4.9 - 调用配置

In [ ]:
response = model.invoke(
  "你好，讲个笑话吧！",
  config={
    "run_name": "joke_generation",  # Custom name for this run
    "tags": ["humor", "demo"],  # Tags for categorization
    "metadata": {"user_id": "123"},  # Custom metadata
  }
)

print(response)

### 4.10 高级配置 - 可配置模型

In [ ]:
# 创建一个可配置的模型实例
# 配置字段包括：model, model_provider, temperature, max_tokens

first_model = init_chat_model(
  model="gpt-4.1-mini",
  temperature=0,
  configurable_fields=("model", "model_provider", "temperature", "max_tokens"),
  config_prefix="first",  # Useful when you have a chain with multiple models
)

# 使用默认配置调用模型
first_model.invoke("what's your name")

In [ ]:
# 在运行时覆盖配置字段
response = first_model.invoke(
  "Tell me a joke",
  config={
    "model_provider": "openai",
    "model": "gpt-5",
    "temperature": 0.7,
    "max_tokens": 1000,
  }
)

In [ ]:
# 以声明方式使用可配置模型

from pydantic import BaseModel, Field

class GetWeather(BaseModel):
  """Get the current weather in a given location"""
  location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


class GetPopulation(BaseModel):
  """Get the current population in a given location"""
  location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

# init_chat_model() 不直接实例化具体模型，而是创建了一个“声明式可配置模型”，
# 你可以在调用时通过 config 参数动态指定模型的配置。
model = init_chat_model(temperature=0)
model_with_tools = model.bind_tools([GetWeather, GetPopulation])

model_with_tools.invoke(
  "what's bigger in 2024 LA or NYC", config={"configurable": {"model": "gpt-4.1-mini"}}
).tool_calls

In [ ]:
model_with_tools.invoke(
    "what's bigger in 2024 LA or NYC",
        config={"configurable": {"model": "gpt-5"}},
).tool_calls